# Script to carry out Metagenomics Data Pre-processing and Assembly

## Rationale

- The script below uses long read nanopore data which has been sequenced from DNA extracted faecal samples. The nanopore reads are DNA and dscDNA from faecal samples of Indonesian Macaques.
- As such the DNA will contain host macaque DNA, plant DNA, bacteria, fungi and viruses. There should also be some protist DNA -- likely Plasmodium, and perhaps some archea. 
- The script follows a basic pre-processing pipeline at first before subsequently branching into a metagenomics focussed pipeline

In [1]:
pwd

'/mnt/shared/scratch/doresegu/private/JCS_MetaGenome_Project/Scripts'

#### Define the parameters needed

In [2]:
import sys
import os
import configparser
import subprocess
import argparse

In [26]:
# read in the config file to get information on the run
#conf=sys.argv[1]
config_file = configparser.ConfigParser()
#config_file.read_file(open(conf))
config_file.read_file(open(r'metagen_config.txt'))

##### Paths

In [27]:
####### PATHS
# get path to the reference genome
REFPATH=config_file.get('PATHS', 'REFPATH')
# get the path to the raw reads
RAWPATH=config_file.get('PATHS', 'RPATH')
# get the path to the output folder
SAVEPATH=config_file.get('PATHS','SPATH')
# get the path to folder holding sub-scripts
SCPTS=config_file.get('PATHS','SCPTS')
# get the path to the basecalled data if available. If not, will be defined later in the script
BASEPATH=config_file.get('PATHS','DPATH')
# get path to the reference GFF file
REFGFF=config_file.get('PATHS','REFGFF')
# get the path to the ONT guppy
ONT=config_file.get('PATHS','ONT')
# get the path to the BUSCO container
busDOCK=config_file.get('PATHS','busDock')

print("Your raw reads are saved in: "+ RAWPATH)
print("Your basecalled reads are saved in: " + BASEPATH)
os.system("mkdir -p " + SAVEPATH)
print("The results will be saved in: " + SAVEPATH)
print("Your reference genome is in: "+ REFPATH + " and its GFF is in: " + REFGFF)

Your raw reads are saved in: /home/doresegu/projects/uosa/Janet_Cox_Singh/Metagenomics/April2021_Optimisation_Sequencing_2samples/RawFiles/Macaque_1_20_04_21/no_sample/20210420_1647_MN17366_FAP72684_88de1f3e/fast5/
Your basecalled reads are saved in: 
The results will be saved in: /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData
Your reference genome is in: /home/doresegu/scratch/private/JCS_MetaGenome_Project/Index/GCF_000956065.1_Mnem_1.0_genomic.fna.gz and its GFF is in: /home/doresegu/scratch/private/JCS_MetaGenome_Project/Index/GCF_000956065.1_Mnem_1.0_genomic.gff.gz


##### Experiment Information

In [28]:
###### get information on the experiment
# what is the name of the experiment
EXPMT=config_file.get('EXPERIMENT INFO', 'EXPMT')
# what sequencing preparation was used
KIT=config_file.get('EXPERIMENT INFO', 'KIT')
KIT2=config_file.get('EXPERIMENT INFO', 'KIT2')
# what barcoding library was used
BCDLIB=config_file.get('EXPERIMENT INFO', 'BCDLIB')
BCDLIB2=config_file.get('EXPERIMENT INFO', 'BCDLIB2')
# what barcodes were used
BRCDES=config_file.get('EXPERIMENT INFO', 'BCODES').split(',')
# what flowcell was used
FLCLL=config_file.get('EXPERIMENT INFO', 'FLOWCELL')
ISOLATES = config_file.get('EXPERIMENT INFO', 'ISOLATES').split(',')
FILT_LENGTH = config_file.get('EXPERIMENT INFO', 'FILT_LENGTH')
FILT_QUAL = config_file.get('EXPERIMENT INFO', 'FILT_QUAL')
print("Your experiment name is: " + EXPMT)
print("Your sequencing kit name is: " + KIT)
print("Your flowcell is: " + FLCLL)

Your experiment name is: April2019
Your sequencing kit name is: SQK-LSK109
Your flowcell is: FLO-MIN106


In [9]:
ISOLATES[0]

'Wu_M_010GEN_FS_dna'

##### Environments

In [11]:
tConv=config_file.get('ENVIRONMENTS', 'tConEnv')
THRDS=config_file.get('PROCESSING', 'THREADS')

##### Basecalling and Demultiplexing

- This step uses Guppy version 6.0.1 via a bash script that is saved in the script folder given in the config file
- Demultiplexing is done using qcat v1.1.0

In [12]:
# set the path to the basecalling script
bascal=SCPTS+"/Basecalling.sh"
# call the basecalling bash script
basecallingScript=['bash',bascal,RAWPATH,SAVEPATH,FLCLL,KIT,ONT]
#print(basecallingScript)
#subprocess.run(basecallingScript)
if BASEPATH=='':
    BASEPATH=SAVEPATH+"/Basecalled"
print("Basecalling is done")

Basecalling is done


In [13]:
# carry out demultiplexing
demCal=SCPTS+"/Demultiplexing.sh"
demultipScrip=['bash',demCal,tConv,BASEPATH,SAVEPATH,THRDS,BCDLIB2]
#subprocess.run(demultipScrip)
print("Demultiplexing done")
print("The demultiplexing outputs are saved in "+SAVEPATH+"/Demultiplexed")

Demultiplexing done
The demultiplexing outputs are saved in /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed


##### Read QC
- This will done with NanoQC, NanoStat and FastQC

In [31]:
def run_QC(SAVEPATH,BRCDES):
    demultiplexed = SAVEPATH + "/Demultiplexed"
    # run nanoStat
    #!mkdir -p "$SAVEPATH/Stats"
    for i in BRCDES:
        statss = SAVEPATH + "/Stats/Raw_Demultiplexed_Reads/" + i
        os.system('mkdir -p ' + statss)
        file = demultiplexed + "/" + i + ".fastq.gz"
        ofile = i + ".txt"
        nanSt = ("NanoStat", "--fastq", file, "--outdir", statss, "-n", ofile)
        runNanSt = ' '.join(nanSt)
        print(runNanSt)
        #subprocess.call(runNanSt, shell=True)
        print('nanoStat complete for ' + i)
        print('Proceeding to nanoQC')
        nanQ = ("nanoQC", "-o", statss, file)
        runNanQ = ' '.join(nanQ)
        print(runNanQ)
        #subprocess.call(runNanQ, shell=True)
        print('nanoQC complete for ' + i)
        print('Proceeding to FastQC')
        fatq = ("fastqc", "-t", THRDS, "-o", statss, file)
        runFatq = ' '.join(fatq)
        print(runFatq)
        #subprocess.call(runFatq, shell=True)
        print('FastQC complete')
run_QC(SAVEPATH,BRCDES)
count = 0
for i in BRCDES:
    isola = ISOLATES[count]
    filtered_path,filtered_stats=filt_qc(SAVEPATH,i,isola)
    print('The raw demultiplexed reads have been successfully filtered and saved in ' + filtered_path)
    print('The QC stats for the filtered reads are saved in ' + filtered_stats)
    count+=1
    print('Please remember that the files are now renamed')
    print(i + ' is now ' + isola)
    print('')

NanoStat --fastq /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz --outdir /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Demultiplexed/barcode01 -n barcode01.txt
nanoStat complete for barcode01
Proceeding to nanoQC
nanoQC -o /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Demultiplexed/barcode01 /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz -l 50
nanoQC complete for barcode01
Proceeding to FastQC
fastqc -t 12 -o /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Demultiplexed/barcode01 /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz
FastQC complete
NanoStat --fastq /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode02.fastq.gz --outdir /home/doresegu/scratch/private/JCS_Meta

In [59]:
def run_QC(SAVEPATH,BRCDE):
    demultiplexed = SAVEPATH + "/Demultiplexed"
    # run nanoStat
    #!mkdir -p "$SAVEPATH/Stats"
    statss = SAVEPATH + "/Stats/Raw_Demultiplexed_Reads/" + BRCDE
    os.system('mkdir -p ' + statss)
    file = demultiplexed + "/" + BRCDE + ".fastq.gz"
    ofile = BRCDE + ".txt"
    nanSt = ("NanoStat", "--fastq", file, "--outdir", statss, "-n", ofile)
    runNanSt = ' '.join(nanSt)
    print(runNanSt)
    #subprocess.call(runNanSt, shell=True)
    print('nanoStat complete for ' + BRCDE)
    print('Proceeding to nanoQC')
    nanQ = ("nanoQC", "-o", statss, file)
    runNanQ = ' '.join(nanQ)
    print(runNanQ)
    #subprocess.call(runNanQ, shell=True)
    print('nanoQC complete for ' + BRCDE)
    print('Proceeding to FastQC')
    fatq = ("fastqc", "-t", THRDS, "-o", statss, file)
    runFatq = ' '.join(fatq)
    print(runFatq)
    #subprocess.call(runFatq, shell=True)
    print('FastQC complete')
    return statss
#run_QC(SAVEPATH,BRCDES)
for i in BRCDES:
    filtered_stats=run_QC(SAVEPATH,i)
    print('The QC stats for the raw reads are saved in ' + filtered_stats)

NanoStat --fastq /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz --outdir /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Raw_Demultiplexed_Reads/barcode01 -n barcode01.txt
nanoStat complete for barcode01
Proceeding to nanoQC
nanoQC -o /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Raw_Demultiplexed_Reads/barcode01 /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz
nanoQC complete for barcode01
Proceeding to FastQC
fastqc -t 12 -o /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Raw_Demultiplexed_Reads/barcode01 /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz
FastQC complete
The QC stats for the raw reads are saved in /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Raw_Demultiplexed_Reads/

Filtering by the defined length you provided
Starting NanoFilt
gunzip -c /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Demultiplexed/barcode01.fastq.gz | NanoFilt -l 50 -q 10 > /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_RawReads/Wu_M_010GEN_FS_dna.fastq.gz
Filtering complete. QC will be done for your new filtered reads
The files have been renamed according to the names provided in the config file
This means that barcode01 has been renamed to Wu_M_010GEN_FS_dna
NanoStat --fastq /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_RawReads/Wu_M_010GEN_FS_dna.fastq.gz --outdir /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Filtered_Demultiplexed_Reads/barcode01_Wu_M_010GEN_FS_dna -n Wu_M_010GEN_FS_dna.txt
nanoStat complete for barcode01
Proceeding to nanoQC
nanoQC -o /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Filtered_Demul

In [54]:
def filt_qc(SAVEPATH,BRCDE,ISOLATE):
    demultiplexed = SAVEPATH + "/Demultiplexed"
    # filtering
    statss = SAVEPATH + "/Stats/Filtered_Demultiplexed_Reads/" + BRCDE + "_" + ISOLATE
    os.system('mkdir -p ' + statss)
    print('Filtering by the defined length you provided')
    print('Starting NanoFilt')
    file = demultiplexed + "/" + BRCDE + ".fastq.gz"
    filt_out = SAVEPATH + "/Filtered_RawReads"
    #os.system('mkdir -p ' + filt_out)
    filt_file = filt_out + "/" + ISOLATE + ".fastq.gz"
    filtSt = ("gunzip -c", file, "| NanoFilt -l", FILT_LENGTH, "-q", FILT_QUAL, ">", filt_file)
    runFiltSt = ' '.join(filtSt)
    #print(runFiltSt)
    #subprocess.call(runFiltSt, shell=True)
    print('Filtering complete. QC will be done for your new filtered reads')
    print('The files have been renamed according to the names provided in the config file')
    print('This means that ' + BRCDE + ' has been renamed to ' + ISOLATE)
    # qc filtered
    ofile = ISOLATES[count] + ".txt"
    nanSt = ("NanoStat", "--fastq", filt_file, "--outdir", statss, "-n", ofile)
    runNanSt = ' '.join(nanSt)
    #print(runNanSt)
    #subprocess.call(runNanSt, shell=True)
    print('nanoStat complete for ' + BRCDE)
    print('Proceeding to nanoQC')
    nanQ = ("nanoQC", "-o", statss, filt_file)
    runNanQ = ' '.join(nanQ)
    #print(runNanQ)
    #subprocess.call(runNanQ, shell=True)
    print('nanoQC complete for ' + BRCDE)
    print('Proceeding to FastQC')
    fatq = ("fastqc", "-t", THRDS, "-o", statss, filt_file)
    runFatq = ' '.join(fatq)
    #print(runFatq)
    #subprocess.call(runFatq, shell=True)
    print('FastQC complete')
    return filt_out, statss
count = 0
for i in BRCDES:
    isola = ISOLATES[count]
    filtered_path,filtered_stats=filt_qc(SAVEPATH,i,isola)
    print('The raw demultiplexed reads have been successfully filtered and saved in ' + filtered_path)
    print('The QC stats for the filtered reads are saved in ' + filtered_stats)
    count+=1
    print('Please remember that the files are now renamed')
    print(i + ' is now ' + isola)
    print('')

Filtering by the defined length you provided
Starting NanoFilt
Filtering complete. QC will be done for your new filtered reads
The files have been renamed according to the names provided in the config file
This means that barcode01 has been renamed to Wu_M_010GEN_FS_dna
nanoStat complete for barcode01
Proceeding to nanoQC
nanoQC complete for barcode01
Proceeding to FastQC
FastQC complete
The raw demultiplexed reads have been successfully filtered and saved in /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_RawReads
The QC stats for the filtered reads are saved in /home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Filtered_Demultiplexed_Reads/barcode01_Wu_M_010GEN_FS_dna
Please remember that the files are now renamed
barcode01 is now Wu_M_010GEN_FS_dna

Filtering by the defined length you provided
Starting NanoFilt
Filtering complete. QC will be done for your new filtered reads
The files have been renamed according to the n

In [65]:
def align(REFPATH,filterd,SAVEPATH,THRDS,ISOLATE):
    file = filterd + "/" + ISOLATE + ".fastq.gz"
    aligned_out = SAVEPATH + "/Filtered_Reads_Aligned_Vs_Reference"
    aligned_files = SAVEPATH + "/Host_Free_Reads"
    statss = SAVEPATH + "/Stats/Alignment_Vs_Reference/" + ISOLATE
    alignCal = SCPTS + "/Alignment.sh"
    alignScrp = ['bash', alignCal, file, aligned_out, aligned_files, THRDS, statss, REFPATH]
    print(alignScrp)
    #subprocess.run(alignScrp)
    #indx_path = SAVEPATH + "/Reference_index.mmi"
    #indx = mp.Aligner(REFPATH, preset="map-ont", n_threads=int(THRDS), fn_idx_out=indx_path)
    #if not indx: 
    #    raise Exception("ERROR: failed to build/load index")
    #for name, seq in mp.fastx_read(file):
    #    print("Aligning sequence " + name + ":")
    #    indx.map(file, n_threads=THRDS)
    print('Alignment done for ' + ISOLATE)
for i in ISOLATES:
    align(REFPATH,filtered_path,SAVEPATH,THRDS,i) #### not working yet

['bash', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/Scripts/Alignment.sh', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_RawReads/Wu_M_010GEN_FS_dna.fastq.gz', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_Reads_Aligned_Vs_Reference', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Host_Free_Reads', '12', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Stats/Alignment_Vs_Reference/Wu_M_010GEN_FS_dna', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/Index/GCF_000956065.1_Mnem_1.0_genomic.fna.gz']
Alignment done for Wu_M_010GEN_FS_dna
['bash', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/Scripts/Alignment.sh', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_RawReads/Wu_M_010GEN_FS_cDNA.fastq.gz', '/home/doresegu/scratch/private/JCS_MetaGenome_Project/JapaneseMacaqueData/Filtered_Reads_Aligned_Vs_

In [61]:
for i in ISOLATES:
    

Wu_M_010GEN_FS_dna
Wu_M_010GEN_FS_cDNA


## Testing section below. All cells should be commented

##### Basecalling and Demultiplexing testing

In [ ]:
#subprocess.run('bash -c "source activate testing"', shell=True)
#sys.stdout = open('test_output.txt', 'w')
#os.system("conda list")
##os.system("qcat --help")
#subprocess.run('bash','qcat','--help')
#sys.stdout.close()
#subprocess.run('bash -c "source deactivate"', shell=True)

#### QC section

In [ ]:
def filt_qc(SAVEPATH,BRCDES):
    demultiplexed = SAVEPATH + "/Demultiplexed"
    count = 0
    for i in BRCDES:
    # filtering
        statss = SAVEPATH + "/Stats/Filtered_Demultiplexed_Reads/" + i + "_" + ISOLATES[count]
        os.system('mkdir -p ' + statss)
        print('Filtering by the defined length you provided')
        print('Starting NanoFilt')
        file = demultiplexed + "/" + i + ".fastq.gz"
        filt_out = SAVEPATH + "/Filtered_RawReads"
        #os.system('mkdir -p ' + filt_out)
        filt_file = filt_out + "/" + ISOLATES[count] + ".fastq.gz"
        filtSt = ("gunzip -c", file, "| NanoFilt -l", FILT_LENGTH, "-q", FILT_QUAL, ">", filt_file)
        runFiltSt = ' '.join(filtSt)
        print(runFiltSt)
        #subprocess.call(runFiltSt, shell=True)
        print('Filtering complete. QC will be done for your new filtered reads')
        print('The files have been renamed according to the names provided in the config file')
        print('This means that ' + i + ' has been renamed to ' + ISOLATES[count])
        # qc filtered
        ofile = ISOLATES[count] + ".txt"
        nanSt = ("NanoStat", "--fastq", filt_file, "--outdir", statss, "-n", ofile)
        runNanSt = ' '.join(nanSt)
        print(runNanSt)
        #subprocess.call(runNanSt, shell=True)
        print('nanoStat complete for ' + i)
        print('Proceeding to nanoQC')
        nanQ = ("nanoQC", "-o", statss, filt_file)
        runNanQ = ' '.join(nanQ)
        print(runNanQ)
        #subprocess.call(runNanQ, shell=True)
        print('nanoQC complete for ' + i)
        print('Proceeding to FastQC')
        fatq = ("fastqc", "-t", THRDS, "-o", statss, filt_file)
        runFatq = ' '.join(fatq)
        print(runFatq)
        #subprocess.call(runFatq, shell=True)
        print('FastQC complete')
        count=+1
        print('')
        print('The raw reads have been successfully filtered and saved in ' + filt_out)
        print('')
        print('The QC stats for the filtered reads are saved in ' + statss)
        print('')
        print('Please remember that the files are now renamed')
        return filt_out

for i in BRCDES:
    filtered_path=filt_qc(SAVEPATH,i)
print('The out is ' + filtered_path)

#### More testing in QC

In [ ]:
#def run_QC(dem_dir,barcode,stats,ofile):
#    stats_dir = os.path.join(stats, "Raw_Demultiplexed_Reads", barcode)
#    if os.path.exists(stats_dir):
#        pass
#    else:
#        os.makedirs(stats_dir)
#    temp = barcode + ".fastq.gz"
#    file_in = os.path.join(dem_dir, temp)
#    ofile = barcode + ".txt"
#    nanSt = ("NanoStat", "--fastq", file_in, "--outdir", stats_dir, "-n", ofile)
#    runNanSt = ' '.join(nanSt)
#    print(runNanSt)
#    #subprocess.call(runNanSt, shell=True)
#    print('nanoStat complete for ' + barcode)
#    print('Proceeding to nanoQC')
#    nanQ = ("nanoQC", "-o", stats_dir, file_in)
#    runNanQ = ' '.join(nanQ)
#    print(runNanQ)
#    subprocess.call(runNanQ, shell=True)
#    print('nanoQC complete for ' + barcode)
#    print('Proceeding to FastQC')
#    fatq = ("fastqc", "-t", str(THREADS), "-o", stats_dir, file_in)
#    runFatq = ' '.join(fatq)
#    print(runFatq)
#    #subprocess.call(runFatq, shell=True)
#    print('FastQC complete')
#    return stats_dir